# 🤖 Sistema de Anotação Automática com Múltiplas LLMs

Este notebook demonstra o uso completo do sistema com datasets do HuggingFace.

## 📋 Conteúdo

1. Setup e Configuração
2. Carregar Dataset do HuggingFace
3. Configurar Modelos LLM
4. Executar Anotação
5. Calcular Consenso
6. Análise Detalhada
7. Visualizações
8. Validação com Ground Truth
9. Exportar Resultados

---

## 1. Setup e Configuração

### Imports

In [ ]:
# Configurar paths
import sys
from pathlib import Path

# Adicionar diretórios ao path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root / 'src' / 'llm_annotation_system'))
sys.path.insert(0, str(project_root / 'src' / 'config'))

print(f"Project root: {project_root}")

NameError: name '__file__' is not defined

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
from pathlib import Path as PathLib

# Sistema de anotação
from src.llm_annotation_system.llm_annotator import LLMAnnotator
from src.llm_annotation_system.consensus_analyzer import ConsensusAnalyzer
from src.utils.visualizer import ConsensusVisualizer

# Datasets HuggingFace
from src.utils.data_loader import (
    load_hf_dataset,
    load_hf_dataset_as_dataframe,
    list_available_datasets,
    discover_dataset_structure
)

# Configurar visualizações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Imports carregados com sucesso!")

ImportError: cannot import name 'BASE_ANNOTATION_PROMPT' from 'config' (unknown location)

### Carregar API Keys

In [1]:
from dotenv import load_dotenv

# Carregar variáveis de ambiente
env_path = project_root / '.env'
load_dotenv(env_path)

# Configurar API keys
api_keys = {
    "openai": os.getenv("OPENAI_API_KEY"),
    "anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "google": os.getenv("GOOGLE_API_KEY"),
}

# Verificar se as keys estão configuradas
for provider, key in api_keys.items():
    if key:
        print(f"✅ {provider}: Configurada")
    else:
        print(f"⚠️  {provider}: NÃO configurada")

print("\n💡 Dica: Configure suas API keys no arquivo .env")

NameError: name 'project_root' is not defined

---

## 2. Carregar Dataset do HuggingFace

### 2.1 Listar Datasets Disponíveis

In [ ]:
# Ver datasets configurados
print("📋 Datasets configurados:")
for dataset_name in list_available_datasets():
    print(f"   • {dataset_name}")

print("\n💡 Configure novos datasets em: src/config/dataset_config.py")

### 2.2 Descobrir Estrutura de um Dataset (Opcional)

Se você quiser explorar um novo dataset antes de configurá-lo:

In [ ]:
# Descomente e ajuste para seu dataset
# discover_dataset_structure("waashk/seu-dataset", num_examples=3)

### 2.3 Carregar Dataset

In [ ]:
# Escolha seu dataset
dataset_name = "exemplo_com_labels"  # AJUSTE para seu dataset

print(f"📦 Carregando dataset: {dataset_name}\n")

# Carregar
texts, categories, ground_truth = load_hf_dataset(dataset_name)

# Informações
print(f"\n📊 Informações do Dataset:")
print(f"   • Total de textos: {len(texts)}")
print(f"   • Categorias: {categories}")
print(f"   • Ground truth disponível: {'Sim ✅' if ground_truth else 'Não'}")

### 2.4 Visualizar Amostra

In [ ]:
# Mostrar primeiros textos
print("📝 Primeiros 5 textos do dataset:\n")
for i in range(min(5, len(texts))):
    print(f"Texto {i+1}:")
    print(f"   {texts[i][:100]}...")
    if ground_truth:
        print(f"   Label: {ground_truth[i]}")
    print()

### 2.5 Análise Exploratória (se houver ground truth)

In [ ]:
if ground_truth:
    # Criar DataFrame
    df_explore = pd.DataFrame({
        'text': texts,
        'label': ground_truth
    })
    
    print("📊 Distribuição de Classes:\n")
    print(df_explore['label'].value_counts())
    
    # Visualizar
    plt.figure(figsize=(10, 5))
    df_explore['label'].value_counts().plot(kind='bar')
    plt.title('Distribuição de Classes no Dataset')
    plt.xlabel('Classe')
    plt.ylabel('Quantidade')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("ℹ️  Sem ground truth - pulando análise exploratória")

---

## 3. Configurar Modelos LLM

### 3.1 Selecionar Modelos

In [ ]:
# Modelos a usar
models = [
    "gpt-4-turbo",
    "gpt-3.5-turbo",
    "claude-3-opus",
    "claude-3-sonnet",
    "gemini-pro",
]

print("🤖 Modelos selecionados:")
for i, model in enumerate(models, 1):
    print(f"   {i}. {model}")

print(f"\n💡 Dica: Para economizar, comece com 3 modelos (ex: gpt-3.5, claude-sonnet, gemini)")

### 3.2 Inicializar Anotador

In [ ]:
# Configurar diretórios
cache_dir = project_root / "cache"
results_dir = project_root / "results"

# Criar diretórios se não existirem
cache_dir.mkdir(exist_ok=True)
results_dir.mkdir(exist_ok=True)

print("🔧 Inicializando anotador...\n")

# Inicializar
annotator = LLMAnnotator(
    models=models,
    categories=categories,
    api_keys=api_keys,
    cache_dir=str(cache_dir),
    results_dir=str(results_dir)
)

print("✅ Anotador configurado!")
print(f"   • Modelos: {len(models)}")
print(f"   • Categorias: {len(categories)}")
print(f"   • Cache: {cache_dir}")
print(f"   • Resultados: {results_dir}")

---

## 4. Executar Anotação

### 4.1 Configurar Parâmetros

In [ ]:
# Parâmetros da anotação
num_repetitions = 3  # Cada LLM anota 3 vezes
test_param_variations = False  # True para testar variações de parâmetros

# Estimativa de custos
total_annotations = len(texts) * len(models) * num_repetitions
estimated_cost = total_annotations * 0.002  # Estimativa aproximada

print("⚙️  Configuração da Anotação:")
print(f"   • Textos: {len(texts)}")
print(f"   • Modelos: {len(models)}")
print(f"   • Repetições por modelo: {num_repetitions}")
print(f"   • Total de anotações: {total_annotations}")
print(f"   • Custo estimado: ${estimated_cost:.2f}")
print(f"\n⚠️  Começando anotação...")

### 4.2 Executar Anotação

In [ ]:
# Anotar dataset
df_annotations = annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions,
    test_param_variations=test_param_variations,
)

print("\n✅ Anotação concluída!")
print(f"   • Shape: {df_annotations.shape}")
print(f"   • Colunas: {len(df_annotations.columns)}")

### 4.3 Visualizar Anotações

In [ ]:
# Mostrar primeiras linhas
print("📋 Primeiras anotações:\n")
display(df_annotations.head())

# Informações das colunas
annotation_cols = [col for col in df_annotations.columns if col.startswith(('gpt', 'claude', 'gemini'))]
print(f"\n📊 Colunas de anotação: {len(annotation_cols)}")
print(f"   Exemplos: {annotation_cols[:5]}")

---

## 5. Calcular Consenso

### 5.1 Executar Análise de Consenso

In [ ]:
print("🧮 Calculando consenso...\n")

# Calcular consenso
df_with_consensus = annotator.calculate_consensus(df_annotations)

print("✅ Consenso calculado!")
print(f"   • Shape: {df_with_consensus.shape}")

# Mostrar novas colunas
consensus_cols = [col for col in df_with_consensus.columns if 'consensus' in col.lower()]
print(f"\n📊 Colunas de consenso adicionadas: {len(consensus_cols)}")
print(f"   {consensus_cols[:5]}")

### 5.2 Estatísticas de Consenso

In [ ]:
# Estatísticas gerais
print("📊 Estatísticas de Consenso:\n")
print(f"   • Consenso médio: {df_with_consensus['consensus_score'].mean():.2%}")
print(f"   • Consenso mediano: {df_with_consensus['consensus_score'].median():.2%}")
print(f"   • Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

# Por nível de consenso
high_consensus = (df_with_consensus['consensus_score'] >= 0.8).sum()
medium_consensus = ((df_with_consensus['consensus_score'] >= 0.6) & 
                    (df_with_consensus['consensus_score'] < 0.8)).sum()
low_consensus = (df_with_consensus['consensus_score'] < 0.6).sum()

print(f"\n📈 Distribuição por Nível:")
print(f"   • Alto consenso (≥80%): {high_consensus} ({high_consensus/len(df_with_consensus):.1%})")
print(f"   • Médio consenso (60-80%): {medium_consensus} ({medium_consensus/len(df_with_consensus):.1%})")
print(f"   • Baixo consenso (<60%): {low_consensus} ({low_consensus/len(df_with_consensus):.1%})")

# Casos problemáticos
problematic = df_with_consensus['is_problematic'].sum()
print(f"\n⚠️  Casos problemáticos: {problematic} ({problematic/len(df_with_consensus):.1%})")

### 5.3 Visualizar Distribuição de Consenso

In [ ]:
# Histograma de consenso
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_with_consensus['consensus_score'], bins=20, edgecolor='black', alpha=0.7)
plt.axvline(0.8, color='green', linestyle='--', label='Alto consenso (80%)')
plt.axvline(0.6, color='orange', linestyle='--', label='Médio consenso (60%)')
plt.xlabel('Score de Consenso')
plt.ylabel('Frequência')
plt.title('Distribuição de Scores de Consenso')
plt.legend()

plt.subplot(1, 2, 2)
levels = ['Alto\n(≥80%)', 'Médio\n(60-80%)', 'Baixo\n(<60%)']
counts = [high_consensus, medium_consensus, low_consensus]
colors = ['green', 'orange', 'red']
plt.bar(levels, counts, color=colors, alpha=0.7)
plt.ylabel('Quantidade')
plt.title('Instâncias por Nível de Consenso')

plt.tight_layout()
plt.show()

---

## 6. Análise Detalhada

### 6.1 Inicializar Analisador

In [ ]:
print("🔬 Inicializando analisador de consenso...\n")

# Criar analisador
analyzer = ConsensusAnalyzer(categories=categories)

# Identificar colunas de consenso
consensus_annotation_cols = [col for col in df_with_consensus.columns 
                              if '_consensus' in col and '_score' not in col]

print(f"✅ Analisador configurado")
print(f"   • Categorias: {len(categories)}")
print(f"   • Colunas de consenso: {len(consensus_annotation_cols)}")

### 6.2 Gerar Relatório Completo

In [ ]:
print("📊 Gerando relatório de consenso...\n")

# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus,
    annotator_cols=consensus_annotation_cols,
    output_dir=str(results_dir)
)

print("\n✅ Relatório gerado!")
print(f"\n📋 Conteúdo do relatório:")
for key in report.keys():
    print(f"   • {key}")

### 6.3 Métricas de Distância

In [ ]:
# Mostrar métricas de distância
print("📏 Métricas de Distância e Concordância:\n")

distance_metrics = report.get('distance_metrics', {})
for metric, value in distance_metrics.items():
    if isinstance(value, (int, float)):
        print(f"   • {metric}: {value:.4f}")

# Interpretação
if 'mean_cohen_kappa' in distance_metrics:
    kappa = distance_metrics['mean_cohen_kappa']
    print(f"\n📊 Interpretação Cohen's Kappa ({kappa:.4f}):")
    if kappa > 0.8:
        print("   ✅ Excelente concordância!")
    elif kappa > 0.6:
        print("   ✅ Boa concordância")
    elif kappa > 0.4:
        print("   ⚠️  Concordância moderada")
    else:
        print("   ❌ Concordância fraca")

### 6.4 Matriz de Concordância

In [ ]:
# Concordância par a par
if 'pairwise_agreement' in report:
    print("🤝 Concordância Par a Par entre Modelos:\n")
    
    pairwise_agreement = report['pairwise_agreement']
    display(pairwise_agreement)
    
    # Estatísticas
    print(f"\n📊 Estatísticas:")
    print(f"   • Concordância média: {pairwise_agreement.values.mean():.2%}")
    print(f"   • Concordância mínima: {pairwise_agreement.values.min():.2%}")
    print(f"   • Concordância máxima: {pairwise_agreement.values.max():.2%}")

### 6.5 Casos Problemáticos

In [ ]:
# Identificar casos problemáticos
problematic_cases = df_with_consensus[df_with_consensus['is_problematic']]

print(f"⚠️  Casos Problemáticos: {len(problematic_cases)}\n")

if len(problematic_cases) > 0:
    # Mostrar alguns exemplos
    print("📝 Exemplos de casos problemáticos:\n")
    for idx in problematic_cases.head(3).index:
        row = df_with_consensus.loc[idx]
        print(f"Texto {idx}:")
        print(f"   {row['text'][:100]}...")
        print(f"   Consenso: {row['consensus_score']:.0%}")
        print(f"   Anotação mais comum: {row['most_common_annotation']}")
        print()
else:
    print("✅ Nenhum caso problemático identificado!")

---

## 7. Visualizações

### 7.1 Inicializar Visualizador

In [ ]:
# Criar diretório de figuras
figures_dir = results_dir / 'figures'
figures_dir.mkdir(exist_ok=True)

print("📊 Inicializando visualizador...\n")

# Criar visualizador
visualizer = ConsensusVisualizer(output_dir=str(figures_dir))

print(f"✅ Visualizador configurado")
print(f"   • Output: {figures_dir}")

### 7.2 Heatmap de Concordância

In [ ]:
# Gerar heatmap
if 'pairwise_agreement' in report:
    print("🔥 Gerando heatmap de concordância...\n")
    
    visualizer.plot_agreement_heatmap(
        report['pairwise_agreement'],
        title="Concordância entre Modelos LLM"
    )
    
    print(f"✅ Heatmap salvo em: {figures_dir}/agreement_heatmap.png")

### 7.3 Distribuição de Consenso

In [ ]:
print("📊 Gerando gráfico de distribuição de consenso...\n")

visualizer.plot_consensus_distribution(df_with_consensus)

print(f"✅ Gráfico salvo em: {figures_dir}/consensus_distribution.png")

### 7.4 Matriz de Confusão

In [ ]:
# Matriz de confusão (se disponível)
if 'disagreement_patterns' in report and 'confusion_matrix' in report['disagreement_patterns']:
    print("🎯 Gerando matriz de confusão...\n")
    
    visualizer.plot_confusion_matrix(
        report['disagreement_patterns']['confusion_matrix']
    )
    
    print(f"✅ Matriz salva em: {figures_dir}/confusion_matrix.png")

### 7.5 Comparação de Modelos

In [ ]:
print("🤖 Gerando comparação de modelos...\n")

visualizer.plot_model_comparison(
    df_with_consensus,
    models=models
)

print(f"✅ Comparação salva em: {figures_dir}/model_comparison.png")

### 7.6 Dashboard Interativo

In [ ]:
print("🌐 Gerando dashboard interativo...\n")

visualizer.create_interactive_dashboard(
    df_with_consensus,
    report
)

dashboard_path = figures_dir / 'interactive_dashboard.html'
print(f"✅ Dashboard salvo em: {dashboard_path}")
print(f"\n💡 Abra no navegador para explorar: {dashboard_path.absolute()}")

---

## 8. Validação com Ground Truth

Se o dataset tem labels, podemos validar a qualidade das anotações.

In [ ]:
if ground_truth:
    print("✅ Ground truth disponível - validando anotações...\n")
    
    # Adicionar ground truth ao DataFrame
    df_with_consensus['ground_truth'] = ground_truth
    
    # Calcular métricas
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    # Accuracy
    accuracy = accuracy_score(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    print(f"📊 VALIDAÇÃO COM GROUND TRUTH\n")
    print(f"   • Accuracy geral: {accuracy:.2%}\n")
    
    # Classification report
    print("📋 Relatório de Classificação:\n")
    print(classification_report(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation'],
        target_names=categories
    ))
    
    # Matriz de confusão
    cm = confusion_matrix(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    # Visualizar matriz de confusão
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=categories, yticklabels=categories)
    plt.title('Matriz de Confusão: Consenso vs Ground Truth')
    plt.ylabel('Ground Truth')
    plt.xlabel('Consenso (Anotação Automática)')
    plt.tight_layout()
    plt.savefig(figures_dir / 'confusion_vs_ground_truth.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✅ Matriz salva em: {figures_dir}/confusion_vs_ground_truth.png")
    
    # Análise por nível de consenso
    print("\n📊 Accuracy por Nível de Consenso:\n")
    
    for threshold, name in [(0.8, 'Alto'), (0.6, 'Médio'), (0.0, 'Baixo')]:
        mask = df_with_consensus['consensus_score'] >= threshold
        if mask.sum() > 0:
            acc = accuracy_score(
                df_with_consensus[mask]['ground_truth'],
                df_with_consensus[mask]['most_common_annotation']
            )
            print(f"   • {name} (≥{threshold:.0%}): {acc:.2%} ({mask.sum()} instâncias)")

else:
    print("ℹ️  Ground truth não disponível - pulando validação")
    print("\n💡 Dica: Use um dataset com labels para validar a qualidade!")

---

## 9. Exportar Resultados

### 9.1 Salvar Datasets

In [ ]:
print("💾 Salvando resultados...\n")

# Criar diretório final
final_dir = results_dir / 'final'
final_dir.mkdir(exist_ok=True)

# 1. Dataset completo
complete_path = final_dir / 'dataset_anotado_completo.csv'
df_with_consensus.to_csv(complete_path, index=False, encoding='utf-8')
print(f"✅ Dataset completo: {complete_path}")

# 2. Alta confiança (≥80%)
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf_path = final_dir / 'alta_confianca.csv'
high_conf.to_csv(high_conf_path, index=False, encoding='utf-8')
print(f"✅ Alta confiança ({len(high_conf)} instâncias): {high_conf_path}")

# 3. Necessita revisão (<80%)
needs_review = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
review_path = final_dir / 'necessita_revisao.csv'
needs_review.to_csv(review_path, index=False, encoding='utf-8')
print(f"✅ Necessita revisão ({len(needs_review)} instâncias): {review_path}")

# 4. Casos problemáticos
if len(problematic_cases) > 0:
    problematic_path = final_dir / 'casos_problematicos.csv'
    problematic_cases.to_csv(problematic_path, index=False, encoding='utf-8')
    print(f"✅ Casos problemáticos ({len(problematic_cases)} instâncias): {problematic_path}")

### 9.2 Sumário em JSON

In [ ]:
import json
from datetime import datetime

# Criar sumário
summary = {
    'dataset': {
        'name': dataset_name,
        'total_instances': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None,
    },
    'configuration': {
        'models': models,
        'num_models': len(models),
        'num_repetitions': num_repetitions,
        'total_annotations': total_annotations,
    },
    'results': {
        'consensus': {
            'mean': float(df_with_consensus['consensus_score'].mean()),
            'median': float(df_with_consensus['consensus_score'].median()),
            'std': float(df_with_consensus['consensus_score'].std()),
        },
        'distribution': {
            'high_consensus': int(high_consensus),
            'medium_consensus': int(medium_consensus),
            'low_consensus': int(low_consensus),
        },
        'problematic_cases': int(problematic),
    },
    'metrics': distance_metrics if 'distance_metrics' in locals() else {},
    'timestamp': datetime.now().isoformat(),
}

# Adicionar validação se disponível
if ground_truth:
    summary['validation'] = {
        'accuracy': float(accuracy),
    }

# Salvar
summary_path = final_dir / 'sumario_experimento.json'
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\n✅ Sumário salvo: {summary_path}")

### 9.3 Resumo Final

In [ ]:
print("\n" + "="*80)
print(" "*25 + "RESUMO FINAL")
print("="*80 + "\n")

print(f"📊 DATASET")
print(f"   • Nome: {dataset_name}")
print(f"   • Total de textos: {len(texts)}")
print(f"   • Categorias: {len(categories)}")

print(f"\n🤖 CONFIGURAÇÃO")
print(f"   • Modelos: {len(models)}")
print(f"   • Repetições: {num_repetitions}")
print(f"   • Total de anotações: {total_annotations}")

print(f"\n🎯 CONSENSO")
print(f"   • Consenso médio: {df_with_consensus['consensus_score'].mean():.2%}")
print(f"   • Alto consenso (≥80%): {high_consensus} ({high_consensus/len(df_with_consensus):.1%})")
print(f"   • Casos problemáticos: {problematic} ({problematic/len(df_with_consensus):.1%})")

if 'mean_cohen_kappa' in distance_metrics:
    print(f"\n📏 MÉTRICAS")
    print(f"   • Cohen's Kappa: {distance_metrics['mean_cohen_kappa']:.4f}")
    if 'fleiss_kappa' in distance_metrics:
        print(f"   • Fleiss' Kappa: {distance_metrics['fleiss_kappa']:.4f}")

if ground_truth:
    print(f"\n✅ VALIDAÇÃO")
    print(f"   • Accuracy vs Ground Truth: {accuracy:.2%}")

print(f"\n📁 ARQUIVOS GERADOS")
print(f"   • Dataset completo: {complete_path.name}")
print(f"   • Alta confiança: {high_conf_path.name}")
print(f"   • Necessita revisão: {review_path.name}")
print(f"   • Dashboard interativo: interactive_dashboard.html")
print(f"   • Sumário JSON: {summary_path.name}")

print(f"\n📂 DIRETÓRIOS")
print(f"   • Resultados: {results_dir}")
print(f"   • Figuras: {figures_dir}")
print(f"   • Final: {final_dir}")

print("\n" + "="*80)
print("✅ ANÁLISE COMPLETA!")
print("="*80 + "\n")

print("🎉 Próximos passos:")
print("   1. Abra o dashboard interativo no navegador")
print("   2. Analise casos de baixo consenso")
print("   3. Revise casos problemáticos")
if ground_truth:
    print("   4. Analise erros vs ground truth")
print("   5. Documente seus achados")
print("   6. Prepare apresentação para orientador")

---

## 🎓 Conclusões e Próximos Passos

### Análise dos Resultados

Analise os resultados considerando:

1. **Taxa de consenso alto**: Quantos % das instâncias têm consenso ≥80%?
2. **Cohen's Kappa**: A concordância entre modelos é boa (>0.6)?
3. **Accuracy** (se tiver ground truth): As anotações estão corretas?
4. **Casos problemáticos**: Por que esses casos são difíceis?

### Questões de Pesquisa

1. Qual threshold de consenso devemos usar?
2. Few-shot learning melhoraria os resultados?
3. Quais modelos são mais consistentes?
4. Vale a pena o custo comparado com anotação humana?

### Melhorias Possíveis

1. **Prompts**: Testar few-shot ou Chain-of-Thought
2. **Modelos**: Adicionar/remover modelos baseado em performance
3. **Parâmetros**: Testar diferentes temperaturas
4. **Estratégias**: Otimizar resolução de conflitos

---

**Boa sorte com sua pesquisa!** 🚀